Ce notebook a pour but d'identiifier de potentiels écarts entre ce qui est récupéré depuis EV Database et la base de données.
Le but originel était du nettoyage mais il pourra permettre de gagner du temps à l'avenir en cas de besoin de nouveau.

In [ ]:
import pandas as pd
import os
from ingestion.ev_database.ev_database import fetch_api_data
from core.sql_utils import get_connection

In [ ]:
# Fetch EV Database

url = os.getenv("EV_DATABASE_URL")
print(f"Fetching data from URL: {url}")

data = fetch_api_data(url)

evdb = set[tuple]((el['Vehicle_Make'], el['Vehicle_Model'], el['Vehicle_Model_Version']) for el in data)
evdb = [
    tuple(
        ("unknown" if x is None else x.lower())
        if isinstance(x, str) or x is None else x
        for x in t
    )
    for t in evdb
]

In [ ]:
# Fetch DB

with get_connection() as conn_rdb, conn_rdb.cursor() as cursor:
    cursor.execute("""
    SELECT make_name, model_name, type
    FROM vehicle_model vm
    LEFT JOIN make ON vm.make_id = make.id
    """)
    db = cursor.fetchall()
    db


db = [el for el in db if el[0]]

In [ ]:
from collections import Counter

# Compare dbs
evdb_set = set([el for el in evdb if el[0] != "tesla"])
db_set_wo_tesla = set([el for el in db if el[0] != "tesla"])
print(f"Nombre de véhicules dans EVDB sans tesla : {len(evdb_set)}")
print(f"Nombre de véhicules dans la DB sans tesla : {len(db_set_wo_tesla)}")

# Set difference
diff_evdb_has = set(evdb_set) - set(db_set_wo_tesla)
diff_db_has = set(db_set_wo_tesla) - set(evdb_set)
print(f"Hors tesla, la database a {len(diff_db_has)} véhicules en plus")
print(f"Hors tesla, EV database a {len(diff_evdb_has)} véhicules en plus")


A date (14/11/2025), ces 23 véhicules sont les utilitaires qu on ne récupère pas encore depuis EV Database &
les véhicules unknwown pour chaque marque. Le tri des autres cas a été fait et vont être antcipés : nettoyage
d'héritage, des véhicules renommés pa EV Database et supprimés par EV Database.

Si la database a plus de 23 véhicules en plus (hors Tesla), c'est soit que des utilitaires ont été ajoutés, soit des unknown
ou alors il y a de nouveaux problèmes à identifier.

In [ ]:
db_wo_tesla_pd = pd.DataFrame(db_set_wo_tesla, columns=["make", "model", "type"])

In [ ]:
# Ci dessous les véhicules en écart qui ne sont pas des unknwown pour chaque marque
pd.set_option("display.max_rows", 100)
diff_db_has_pd = pd.DataFrame(diff_db_has, columns=["make", "model", "type"])
diff_db_has_pd[~((diff_db_has_pd['model'] == "unknown") & (diff_db_has_pd['type'] == "unknown"))].sort_values(by="make").head(100)